In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [2]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [3]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler


In [4]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [5]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    lstm_loss_history = loss_history.history['loss']
    
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    plt.plot(y_test, label='Actual')
    plt.plot(lstm_predictions, label='Predicted')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig(f'Ethereum_Models/actual_vs_predicted_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.png')
    plt.close()
    
    rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history.history['loss'], label='Training Loss')
    plt.plot(loss_history.history['val_loss'], label='Validation Loss')
    plt.title('Epoch Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'Ethereum_Models/loss_curve_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.png')
    plt.close()
    
    return model, rmse


In [6]:
symbol = 'ETH-USD'
start_date = '2020-01-01'
end_date = '2024-01-01'

In [7]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [8]:
epochs_range = [50, 100]
batch_sizes = [4,8]
sequence_lengths = [25,60,80]
units_range = [50, 100]
layers_range = [2, 3]

In [9]:
results = []

In [10]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_lstm_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})


Training model with epochs=50, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/50

263/263 [==============================] - 35s 67ms/step - loss: 0.0069 - val_loss: 3.6463e-04
Epoch 2/50


C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


263/263 [==============================] - 14s 53ms/step - loss: 0.0032 - val_loss: 2.8510e-04
Epoch 3/50
263/263 [==============================] - 14s 52ms/step - loss: 0.0020 - val_loss: 2.3408e-04
Epoch 4/50
263/263 [==============================] - 13s 49ms/step - loss: 0.0016 - val_loss: 1.9299e-04
Epoch 5/50
263/263 [==============================] - 12s 47ms/step - loss: 0.0014 - val_loss: 2.0785e-04
Epoch 6/50
263/263 [==============================] - 14s 52ms/step - loss: 0.0013 - val_loss: 7.1698e-04
Epoch 7/50
263/263 [==============================] - 14s 52ms/step - loss: 0.0010 - val_loss: 1.3595e-04
Epoch 8/50
263/263 [==============================] - 17s 66ms/step - loss: 9.6669e-04 - val_loss: 1.2462e-04
Epoch 9/50
263/263 [==============================] - 18s 70ms/step - loss: 9.0353e-04 - val_loss: 1.1794e-04
Epoch 10/50
263/263 [==============================] - 16s 62ms/step - loss: 8.6542e-04 - val_loss: 6.9409e-04
Epoch 11/50
263/263 [=======================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


263/263 [==============================] - 16s 62ms/step - loss: 0.0040 - val_loss: 7.3110e-04
Epoch 3/50
263/263 [==============================] - 16s 62ms/step - loss: 0.0030 - val_loss: 2.9659e-04
Epoch 4/50
263/263 [==============================] - 15s 58ms/step - loss: 0.0022 - val_loss: 7.7904e-04
Epoch 5/50
263/263 [==============================] - 15s 58ms/step - loss: 0.0018 - val_loss: 6.8288e-04
Epoch 6/50
263/263 [==============================] - 17s 66ms/step - loss: 0.0014 - val_loss: 4.3133e-04
Epoch 7/50
263/263 [==============================] - 17s 65ms/step - loss: 0.0012 - val_loss: 9.3215e-04
Epoch 8/50
263/263 [==============================] - 18s 67ms/step - loss: 0.0012 - val_loss: 1.5790e-04
Epoch 9/50
263/263 [==============================] - 16s 61ms/step - loss: 9.5666e-04 - val_loss: 1.2827e-04
Epoch 10/50
263/263 [==============================] - 16s 60ms/step - loss: 0.0013 - val_loss: 1.5606e-04
Epoch 11/50
263/263 [==============================]

...

132/132 [==============================] - 21s 159ms/step - loss: 0.0042 - val_loss: 0.0019
Epoch 3/100
132/132 [==============================] - 21s 161ms/step - loss: 0.0037 - val_loss: 5.5296e-04
Epoch 4/100
132/132 [==============================] - 21s 161ms/step - loss: 0.0036 - val_loss: 3.6856e-04
Epoch 5/100
132/132 [==============================] - 21s 160ms/step - loss: 0.0021 - val_loss: 0.0012
Epoch 6/100
132/132 [==============================] - 21s 158ms/step - loss: 0.0021 - val_loss: 8.6205e-04
Epoch 7/100
132/132 [==============================] - 21s 158ms/step - loss: 0.0016 - val_loss: 9.0448e-04
Epoch 8/100
132/132 [==============================] - 21s 160ms/step - loss: 0.0016 - val_loss: 4.9206e-04
Epoch 9/100
132/132 [==============================] - 21s 163ms/step - loss: 0.0013 - val_loss: 3.2827e-04
Epoch 10/100
132/132 [==============================] - 21s 159ms/step - loss: 0.0013 - val_loss: 2.2752e-04
Epoch 11/100
132/132 [=========================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


132/132 [==============================] - 18s 137ms/step - loss: 0.0037 - val_loss: 0.0013
Epoch 3/100
132/132 [==============================] - 18s 138ms/step - loss: 0.0021 - val_loss: 3.5901e-04
Epoch 4/100
132/132 [==============================] - 18s 139ms/step - loss: 0.0021 - val_loss: 2.8763e-04
Epoch 5/100
132/132 [==============================] - 18s 138ms/step - loss: 0.0012 - val_loss: 5.0749e-04
Epoch 6/100
132/132 [==============================] - 18s 137ms/step - loss: 0.0012 - val_loss: 8.4294e-04
Epoch 7/100
132/132 [==============================] - 18s 137ms/step - loss: 0.0013 - val_loss: 2.2030e-04
Epoch 8/100
132/132 [==============================] - 18s 138ms/step - loss: 9.9682e-04 - val_loss: 0.0021
Epoch 9/100
132/132 [==============================] - 18s 138ms/step - loss: 0.0012 - val_loss: 9.1773e-04
Epoch 10/100
132/132 [==============================] - 19s 141ms/step - loss: 8.2740e-04 - val_loss: 1.7557e-04
Epoch 11/100
132/132 [=================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


132/132 [==============================] - 25s 189ms/step - loss: 0.0037 - val_loss: 5.6350e-04
Epoch 3/100
132/132 [==============================] - 23s 172ms/step - loss: 0.0028 - val_loss: 6.2226e-04
Epoch 4/100
132/132 [==============================] - 23s 176ms/step - loss: 0.0022 - val_loss: 3.6313e-04
Epoch 5/100
132/132 [==============================] - 23s 175ms/step - loss: 0.0019 - val_loss: 2.9163e-04
Epoch 6/100
132/132 [==============================] - 23s 175ms/step - loss: 0.0016 - val_loss: 2.4196e-04
Epoch 7/100
132/132 [==============================] - 23s 174ms/step - loss: 0.0017 - val_loss: 2.0268e-04
Epoch 8/100
132/132 [==============================] - 23s 172ms/step - loss: 0.0013 - val_loss: 3.4084e-04
Epoch 9/100
132/132 [==============================] - 23s 173ms/step - loss: 0.0010 - val_loss: 5.0379e-04
Epoch 10/100
132/132 [==============================] - 23s 174ms/step - loss: 7.9387e-04 - val_loss: 1.6319e-04
Epoch 11/100
132/132 [=============

In [11]:
for result in results:
    print(result)

{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 127.08885006285385}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 45.149463312730155}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 43.78698928385512}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 46.17269548700949}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 2, 'rmse': 44.61902077438216}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 3, 'rmse': 42.83370651158674}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 2, 'rmse': 43.922054015168726}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 3, 'rmse': 42.36064631238659}
{'epochs': 50, 'batch_size': 4, 'sequence_length': 80, 'units': 50, 'layers': 2, 'rmse': 43.41727548299754}
{'epochs': 50, 'batch